# Hyperparameter Tuning with Optuna (XGBoost) for 30 Days of ML Challenge


## 1. Import libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For encoding categorical variables
from sklearn import preprocessing

# For the construction of the pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# For training the XGBoost model
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# For the hyperparameter tuning
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_20_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_50_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_15_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_30_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_5_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_25_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_10_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_12_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_60_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_40_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_3_folds.csv
/kaggle/input/train-folds-k-folds-30-days-of-ml/train_folds_6_folds.csv
/kaggle/input/train-folds-30-days-of-ml/train_folds.csv
/kaggle/input/30-days-of-ml/sample_submission.csv
/kaggle/input/30-days

In [2]:
# Load the training and test data. 
X_full = pd.read_csv("../input/train-folds-k-folds-30-days-of-ml/train_folds_3_folds.csv")
X_test_full = pd.read_csv("../input/30-days-of-ml/test.csv")

In [3]:
# We select all features except "id", "target" and "kfold", as these are not predictors of our target.
useful_features = [c for c in X_full.columns if c not in ("id", "target", "kfold")]

# Select numerical columns by data type, not by column name
num_cols = [col for col in X_full[useful_features] if X_full[col].dtype in ['int64', 'float64']]

# We select categorical columns. Note that we dropped the cardinality check.
object_cols = [col for col in useful_features if 'cat' in col]

# We build X_test out of X_test_full, but only selecting the useful features.
X_test = X_test_full[useful_features]

# Next up, we set up the for loop which will perform the target encoding:
for col in object_cols: 
    temp_X_full = [] # We create a temporary list to store the dataframes.
    temp_test_feature = None # We create a temporary feature for the test set.
    
    for fold in range(3): # We loop across all folds
        X_train = X_full[X_full.kfold != fold].reset_index(drop=True) 
        X_valid = X_full[X_full.kfold == fold].reset_index(drop=True) 
        feat = X_train.groupby(col)["target"].agg("mean") # We group the columns by target, and then we get the mean value of the values in "target" column.
        feat = feat.to_dict() # We convert the dataframe into a dictionary.
        X_valid.loc[:, f"tar_enc_{col}"] = X_valid[col].map(feat) # We map the mean values to a new column in X_valid.
        temp_X_full.append(X_valid) # We append X_valid to our temporary list.
        
        if temp_test_feature is None: # If we don't have a temp_test_feature...
            temp_test_feature = X_test[col].map(feat) # ...we assign it this value.
            
        else: # If its not None, (for folds above 0)...
            temp_test_feature = temp_test_feature + X_test[col].map(feat) # ...add to it the present value.
            
    temp_test_feature = temp_test_feature/5 # We divide by the number of folds to get the average.
    X_test.loc[:, f"tar_enc_{col}"] = temp_test_feature # We assign the temp_test_feat value to a new column.
    X_full = pd.concat(temp_X_full) # We build the new X_full dataframe with the new target encoding columns.

In [4]:
def run(trial):
    scores = []
    for fold in range(3):
        learning_rate = trial.suggest_float("learning_rate", 0.01, 0.25, log=True)
        reg_lambda = trial.suggest_loguniform("reg_lambda", 0.00000001, 100.0)
        reg_alpha = trial.suggest_loguniform("reg_alpha", 0.00000001, 100.0)
        subsample = trial.suggest_float("subsample", 0.1, 1.0)
        colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
        max_depth = trial.suggest_int("max_depth", 1, 7)

        X_train = X_full[X_full.kfold != fold].reset_index(drop=True) # We set the training data to be all folds different from the current fold number in the loop. We also reset the indices.
        X_valid = X_full[X_full.kfold == fold].reset_index(drop=True) # The validation data is the current fold number in the loop. We also reset the indices.

        y_train = X_train.target # We set the training target equal to the target in the training set. This has to be done every iteration (as the fold and the data changes).
        y_valid = X_valid.target # We set the validation target equal to the target in the validation set. This has to be done every iteration (as the fold and the data changes).

        X_train = X_train[useful_features] # We set our training data to be the previously defined useful features of X_train.
        X_valid = X_valid[useful_features] # We set our validation data to be the previously defined useful features of X_valid.

        X_train.head()
        
        # Remove categorical columns (will replace with one-hot encoding)
        num_X_train = X_train.drop(object_cols, axis=1)
        num_X_valid = X_valid.drop(object_cols, axis=1)
        
        num_X_train.head()
        
        # Remove categorical columns (will replace with one-hot encoding)
        OH_X_train = X_train.drop(num_cols, axis=1)
        OH_X_valid = X_valid.drop(num_cols, axis=1)
        
        OH_X_train.head()
        
        # We set up one-hot encoding:
        one_hot_encoder = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
        OH_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(OH_X_train))
        OH_cols_valid = pd.DataFrame(one_hot_encoder.transform(OH_X_valid))
        #OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

        # One-hot encoding removed index; put it back
        OH_cols_train.index = X_train.index
        OH_cols_valid.index = X_valid.index
        #OH_cols_test.index = X_test.index

        # Stardardization preprocessing for numerical data
        numerical_transformer = preprocessing.StandardScaler()
        num_cols_train = pd.DataFrame(numerical_transformer.fit_transform(num_X_train))
        num_cols_valid = pd.DataFrame(numerical_transformer.transform(num_X_valid))
        #num_X_test = numerical_transformer.transform(X_test[num_cols])

        # Add one-hot encoded columns to processed numerical features
        new_X_train = pd.concat([num_cols_train, OH_cols_train], axis=1)
        new_X_valid = pd.concat([num_cols_valid, OH_cols_valid], axis=1)
        
        new_X_train = new_X_train.loc[:,~new_X_train.columns.duplicated()]
        new_X_valid = new_X_valid.loc[:,~new_X_valid.columns.duplicated()]

        model = XGBRegressor(
            random_state=0,
            tree_method="gpu_hist",
            gpu_id=0,
            predictor="gpu_predictor",
            n_estimators=25000,
            learning_rate=learning_rate,
            reg_lambda=reg_lambda,
            reg_alpha=reg_alpha,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            max_depth=max_depth,
        )

        model.fit(new_X_train, y_train, early_stopping_rounds=270, eval_set=[(new_X_valid, y_valid)], verbose=1000)   
        preds_valid = model.predict(new_X_valid)
        rmse = mean_squared_error(y_valid, preds_valid, squared=False)
        scores.append(rmse)
        
    return np.mean(scores)

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(run, n_trials=100)

[I 2021-08-31 05:09:46,271] A new study created in memory with name: no-name-9f25b184-d04d-4b8d-a0ae-b5c758fd13ac


[0]	validation_0-rmse:7.18715
[627]	validation_0-rmse:0.72474
[0]	validation_0-rmse:7.18993
[636]	validation_0-rmse:0.72714
[0]	validation_0-rmse:7.19286
[678]	validation_0-rmse:0.72454


[I 2021-08-31 05:10:07,476] Trial 0 finished with value: 0.724407056065476 and parameters: {'learning_rate': 0.07633107719281673, 'reg_lambda': 0.377181416123027, 'reg_alpha': 8.349933266720312e-06, 'subsample': 0.4786629442438145, 'colsample_bytree': 0.575934587515768, 'max_depth': 6}. Best is trial 0 with value: 0.724407056065476.


[0]	validation_0-rmse:6.28046
[1000]	validation_0-rmse:0.71967
[1191]	validation_0-rmse:0.71969
[0]	validation_0-rmse:6.28311
[1000]	validation_0-rmse:0.72229
[1499]	validation_0-rmse:0.72235
[0]	validation_0-rmse:6.28621
[1000]	validation_0-rmse:0.71931
[1331]	validation_0-rmse:0.71943


[I 2021-08-31 05:10:29,975] Trial 1 finished with value: 0.7203890310167161 and parameters: {'learning_rate': 0.19416823589151702, 'reg_lambda': 2.2464255197617606e-05, 'reg_alpha': 72.62839454568923, 'subsample': 0.8983489872481278, 'colsample_bytree': 0.19178879027795978, 'max_depth': 4}. Best is trial 1 with value: 0.7203890310167161.


...

[I 2021-08-31 06:50:30,747] Trial 97 finished with value: 0.7195883294931734 and parameters: {'learning_rate': 0.02579705377341659, 'reg_lambda': 4.935775600533873e-08, 'reg_alpha': 36.227757710759924, 'subsample': 0.9166297230710633, 'colsample_bytree': 0.19155383571661494, 'max_depth': 4}. Best is trial 66 with value: 0.719135219873274.


[0]	validation_0-rmse:7.49220
[1000]	validation_0-rmse:0.72380
[2000]	validation_0-rmse:0.72051
[3000]	validation_0-rmse:0.71926
[4000]	validation_0-rmse:0.71877
[5000]	validation_0-rmse:0.71861
[6000]	validation_0-rmse:0.71856
[6123]	validation_0-rmse:0.71856
[0]	validation_0-rmse:7.49475
[1000]	validation_0-rmse:0.72645
[2000]	validation_0-rmse:0.72329
[3000]	validation_0-rmse:0.72209
[4000]	validation_0-rmse:0.72171
[5000]	validation_0-rmse:0.72153
[5513]	validation_0-rmse:0.72154
[0]	validation_0-rmse:7.49750
[1000]	validation_0-rmse:0.72292
[2000]	validation_0-rmse:0.71983
[3000]	validation_0-rmse:0.71877
[4000]	validation_0-rmse:0.71837
[5000]	validation_0-rmse:0.71828
[5070]	validation_0-rmse:0.71829


[I 2021-08-31 06:51:27,898] Trial 98 finished with value: 0.7194448725786898 and parameters: {'learning_rate': 0.03673985244339761, 'reg_lambda': 5.82325694028999e-07, 'reg_alpha': 8.910487547631355, 'subsample': 0.8099018313882888, 'colsample_bytree': 0.16432612236465452, 'max_depth': 3}. Best is trial 66 with value: 0.719135219873274.


[0]	validation_0-rmse:7.60887
[1000]	validation_0-rmse:0.72301
[2000]	validation_0-rmse:0.72075
[3000]	validation_0-rmse:0.72025
[3342]	validation_0-rmse:0.72024
[0]	validation_0-rmse:7.61140
[1000]	validation_0-rmse:0.72548
[2000]	validation_0-rmse:0.72340
[3000]	validation_0-rmse:0.72296
[3098]	validation_0-rmse:0.72297
[0]	validation_0-rmse:7.61418
[1000]	validation_0-rmse:0.72216
[2000]	validation_0-rmse:0.71997
[3000]	validation_0-rmse:0.71977
[3054]	validation_0-rmse:0.71977


[I 2021-08-31 06:52:16,017] Trial 99 finished with value: 0.7209409541944917 and parameters: {'learning_rate': 0.021589135312833376, 'reg_lambda': 2.035174811533607e-07, 'reg_alpha': 8.775072384908451e-06, 'subsample': 0.21704574828574408, 'colsample_bytree': 0.1374284143752392, 'max_depth': 5}. Best is trial 66 with value: 0.719135219873274.


In [6]:
study.best_params

{'learning_rate': 0.03313934079213014,
 'reg_lambda': 7.795455194937734e-07,
 'reg_alpha': 11.375472681850685,
 'subsample': 0.8202458209691414,
 'colsample_bytree': 0.10071397127578051,
 'max_depth': 3}

In [7]:
study.best_params

{'learning_rate': 0.03313934079213014,
 'reg_lambda': 7.795455194937734e-07,
 'reg_alpha': 11.375472681850685,
 'subsample': 0.8202458209691414,
 'colsample_bytree': 0.10071397127578051,
 'max_depth': 3}